<a href="https://colab.research.google.com/github/lizawang/PCL/blob/main/pcl_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd drive/MyDrive/PCL/
%ls

/content/drive/MyDrive/PCL
10-langs-subtask1_all/            raw.pickle
5nn_predictions.csv               README.txt
backtranslation_subtask2/         rebuilddata.py
data1                             ref/
data1_augmented.csv               res/
data1.pickle                      sentiment_scoring.py
data2.pickle                      task4_test.tsv
DeepMoji/                         ten-langs-regex-cleaned-binary-label.csv
dev_semeval_parids-labels.csv     thePoorerTheMerrier/
dontpatronizeme_categories.tsv    tptm.csv
dontpatronizeme_pcl.tsv           train_semeval_parids-labels.csv
dont_patronize_me.py              trn_5nn_predictions.csv
emoji_feature_vectors_trn_x1.csv  trndf1.csv
emoji_feature_vectors_tst_x1.csv  trndf2.csv
evaluation.py                     trn_encoded_with_senti_df1
model_graph.png                   trn_x1_augmented
modelsForStack/                   trn_x1_original
pcl.ipynb                         tstdf1.csv
pcl-task1-augment-revised.ipynb   tstdf2.csv
pcl-task2-aug

In [ ]:
!pip install emoji
from dont_patronize_me import DontPatronizeMe as dpm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from DeepMoji.deepmoji.attlayer import AttentionWeightedAverage
from DeepMoji import main_emoji as emoji
from sklearn.utils import class_weight
import pickle
from sklearn.metrics import f1_score, classification_report

     |████████████████████████████████| 174 kB 5.3 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=efedb63f34446195ff07451ad5d4586e77c639d2501c6c6fc40e4ed30535ba45
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


# Task 1

## Load data:

In [ ]:
trn_file1 = "dontpatronizeme_pcl.tsv"
trn_file2 = "dontpatronizeme_categories.tsv"
tst_path = "task4_test.tsv"
dpm_o = dpm('.', tst_path)
dpm_o.load_task1()
dpm_o.load_task2()
dpm_o.load_test()
data1 = dpm_o.train_task1_df
data2 = dpm_o.train_task2_df
data_tst = dpm_o.test_set

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


In [ ]:
data1 = data1[data1["text"] != ""] #remove the empty text row 8639 with label 0
for i, s in enumerate(data1["text"]):
  if s=="":
    print(i)

In [ ]:
## get test set from the origninal dataset
x1, y1 = data1["text"], data1["label"]
_, tst_x1, _, tst_y1 = train_test_split(x1, y1, test_size=0.2, random_state=36)
tst_txt_df = pd.DataFrame(tst_x1)
tst_txt_df

,text
4988,"in fact , romance by far out-produces and out-..."
3053,"in the past , dorel has had moments of brillia..."
1984,a man who reached out to a community facebook ...
3547,other westerners who met similar fate to foley...
4891,"""the home affairs department must explain why ..."
...,...
7193,"morris , who is blind , made the call in the s..."
8842,"earlier , submitting the report on the last mo..."
6110,""""""" the broad part of officials and the masses..."
5848,it seems some chinese immigrants come across a...


In [ ]:
## get the training set from the augmented dataset
trndf1 = pd.read_csv("ten-langs-regex-cleaned-binary-label.csv", encoding="utf-8")
trndf1 = trndf1.dropna() #drop the nan sentence on 733 row
#trndf1 = trndf1.sample(frac=1) # shuffle the augmented data. 

In [ ]:
trn_x1 = trndf1["text"]
trn_txt_df = trndf1[['text']]
trn_y1 = trndf1["label"]
trn_txt_df

,text
0,"""uk independence party ( ukip ) leader nigel f..."
1,nasruddin had asked of the measures taken by t...
2,u.s. border patrol agents walk fences on the m...
3,at this point in labour 's leadership battle b...
4,"tina anderson , clerk of the aleutians east bo..."
...,...
15151,""""" My daughter , who was a physiotherapist , w..."
15152,The Turkish ambassador to tanzania ali davutog...
15153,"Finally , there seems to be a glimmer of hope ..."
15154,""" The current government is dedicated to solvi..."


## Preprocessing (clean and tokenize the data):

In [ ]:
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def tok_vec(types_num, trn_x, tst_x, matrix=False):
  stopws = set(stopwords.words("english"))
  trn_x = [" ".join([word for word in s.split() if word not in stopws]) for s in trn_x]
  tst_x = [" ".join([word for word in s.split() if word not in stopws]) for s in tst_x]
  tokenizer = Tokenizer(num_words= types_num, filters = '@[\\]^{|}~\t\n)')   #keep the punctuations
  tokenizer.fit_on_texts(trn_x)
  trn_x_encoded = tokenizer.texts_to_sequences(trn_x)
  tst_x_encoded = tokenizer.texts_to_sequences(tst_x)
  #re = [(k, v) for k, v in sorted(tokenizer.word_counts.items(), key=lambda item: item[1], reverse=True)]
  max_len = int(np.max([len(t) for t in trn_x_encoded])/4)
  trn_x_encoded = np.array(pad_sequences(trn_x_encoded, maxlen=max_len, padding="post", truncating="post"))
  tst_x_encoded = np.array(pad_sequences(tst_x_encoded, maxlen=max_len, padding="post", truncating="post"))
  return trn_x_encoded, tst_x_encoded, max_len

In [ ]:
types_num = 10000
trn_x_encoded1, tst_x_encoded1, max_len = tok_vec(types_num, trn_x1, tst_x1)
trn_x_encoded1.shape, tst_x_encoded1.shape, max_len

((15155, 124), (2094, 124), 124)

In [ ]:
trn_y_encoded1 = np.array(trn_y1)
tst_y_encoded1 = np.array(tst_y1)
trn_y_encoded1.shape, tst_y_encoded1, np.sum([l for l in trn_y_encoded1 if l==1])

((15155,), array([0, 0, 0, ..., 0, 0, 1]), 7569)

In [ ]:
# the class labels are far from balanced, so we use class_weight
from sklearn.utils import class_weight
class_weights = dict(zip(np.unique(trn_y_encoded1), class_weight.compute_class_weight(class_weight='balanced', 
                                                  classes=np.unique(trn_y_encoded1), y=trn_y_encoded1)))
class_weights, np.unique(trn_y_encoded1)

({0: 0.9988795148958608, 1: 1.001123001717532}, array([0, 1]))

### Get sentiment scoring features

In [ ]:
!pip install vaderSentiment
!pip install flair
from sentiment_scoring import senti_score
from scipy.sparse import hstack, csr_matrix
trn_x_sc1 = np.array(senti_score(trn_x1, "vader"))
tst_x_sc1 = np.array(senti_score(tst_x1, "vader"))
tst_x_sc1

     |████████████████████████████████| 125 kB 5.3 MB/s 
     |████████████████████████████████| 322 kB 5.1 MB/s 
     |████████████████████████████████| 788 kB 43.5 MB/s 
     |████████████████████████████████| 1.2 MB 49.6 MB/s 
     |████████████████████████████████| 981 kB 52.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 48 kB 4.8 MB/s 
     |████████████████████████████████| 19.7 MB 1.3 MB/s 
     |████████████████████████████████| 67 kB 4.3 MB/s 
     |████████████████████████████████| 64 kB 2.8 MB/s 
     |████████████████████████████████| 3.5 MB 56.7 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 596 kB 65.5 MB/s 
     |████████████████████████████████| 895 kB 52.2 MB/s 
     |████████████████████████████████| 6.8 MB 45.9 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9

array([[ 0.    ,  0.878 ,  0.122 ,  0.802 ],
       [ 0.179 ,  0.776 ,  0.044 , -0.8169],
       [ 0.072 ,  0.722 ,  0.206 ,  0.6128],
       ...,
       [ 0.    ,  0.88  ,  0.12  ,  0.836 ],
       [ 0.05  ,  0.777 ,  0.174 ,  0.6108],
       [ 0.27  ,  0.671 ,  0.059 , -0.886 ]])

### Stack sentiment scores to data

In [ ]:
trn_x_encoded_with_senti = np.hstack((trn_x_encoded1, trn_x_sc1))
tst_x_encoded_with_senti = np.hstack((tst_x_encoded1, tst_x_sc1))
max_len = max_len + trn_x_sc1.shape[1]
tst_x_encoded_with_senti.shape, max_len

((2094, 128), 128)

### Get predictions from emoji model

In [ ]:
pred_emoji = emoji.emoji_prediction(trn_txt_df, maxlen = max_len)
trn_x1_emoji = pd.DataFrame(pred_emoji)
trn_x1_emoji 

Tokenizing using dictionary from /content/drive/MyDrive/PCL/DeepMoji/model/vocabulary.json
Loading model from /content/drive/MyDrive/PCL/DeepMoji/model/deepmoji_weights.hdf5.
Model: "DeepMoji"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 128, 256)     12800000    ['input_1[0][0]']                
                                                                                                  
 activation (Activation)        (None, 128, 256)     0           ['embedding[0][0]']              
                                                                                                  
 bi_lstm_0 (Bid

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75
0,"""uk independence party ( ukip ) leader nigel f...",0.271587,33,40,21,24,17,0.101379,0.052402,0.051799,0.034831,0.031176,0.009574,0.008855,0.001965,0.004861,0.003309,0.007202,0.017657,0.015409,0.017025,0.006392,0.007743,0.019161,0.008443,0.023588,0.003004,0.005525,0.015897,0.031176,0.004422,0.014693,0.010217,0.051799,0.016837,0.003170,0.034831,0.025898,0.002634,0.012750,0.007789,0.003009,0.026825,0.016309,0.017591,0.101379,0.018806,0.007399,0.003705,0.008203,0.005918,0.004410,0.052402,0.028585,0.010126,0.006659,0.008151,0.013465,0.017956,0.024310,0.005521,0.005705,0.026003,0.006161,0.007436,0.025784,0.019781,0.015508,0.004166,0.016260,0.023923,0.010627,0.006184,0.031141,0.013369,0.015399
1,nasruddin had asked of the measures taken by t...,0.386198,32,55,25,19,12,0.114602,0.080233,0.072154,0.069575,0.049633,0.032743,0.030485,0.006344,0.010074,0.000554,0.009102,0.005517,0.004150,0.001546,0.005344,0.008794,0.001366,0.049633,0.004239,0.006813,0.001817,0.002267,0.004065,0.000458,0.069575,0.020581,0.005315,0.038028,0.000367,0.000549,0.072154,0.002978,0.014882,0.018154,0.008185,0.005549,0.006741,0.114602,0.019336,0.016841,0.008539,0.001186,0.033781,0.015510,0.012618,0.013624,0.014517,0.022599,0.014966,0.011915,0.015102,0.008410,0.001418,0.000480,0.009700,0.005437,0.030764,0.016011,0.011667,0.007005,0.080233,0.010448,0.004547,0.016290,0.001295,0.000465,0.003092,0.028192,0.001037
2,u.s. border patrol agents walk fences on the m...,0.380890,44,32,55,33,25,0.110542,0.103626,0.070165,0.058596,0.037961,0.006054,0.009758,0.001800,0.001695,0.000844,0.003678,0.006354,0.014457,0.005670,0.003399,0.007277,0.006871,0.019221,0.009907,0.001527,0.002225,0.011051,0.005582,0.001378,0.024029,0.013830,0.020753,0.033681,0.001240,0.011153,0.037961,0.001963,0.011187,0.005203,0.002416,0.019260,0.027775,0.103626,0.058596,0.011709,0.002807,0.005045,0.020905,0.004637,0.007657,0.012628,0.019147,0.012810,0.012799,0.110542,0.010352,0.004949,0.009861,0.001406,0.002798,0.026327,0.009647,0.015259,0.029581,0.036932,0.070165,0.007772,0.005886,0.011288,0.004564,0.004456,0.004483,0.025941,0.006227
3,at this point in labour 's leadership battle b...,0.488369,44,32,55,33,22,0.157494,0.144506,0.086157,0.050563,0.049649,0.002976,0.007366,0.000712,0.000473,0.000240,0.004099,0.004310,0.007518,0.004589,0.002291,0.004177,0.001112,0.014764,0.009265,0.004968,0.001307,0.004805,0.004067,0.000623,0.017007,0.007788,0.024762,0.049649,0.000548,0.008319,0.034932,0.000787,0.012882,0.003286,0.000903,0.013474,0.008884,0.144506,0.050563,0.012760,0.004979,0.000920,0.024595,0.004142,0.003030,0.014078,0.047849,0.004245,0.009064,0.157494,0.009242,0.009085,0.009701,0.000337,0.001469,0.015653,0.004296,0.008730,0.014646,0.012520,0.086157,0.005241,0.011222,0.033331,0.005598,0.001965,0.008938,0.016112,0.004652
4,"tina anderson , clerk of the aleutians east bo...",0.316807,32,33,55,21,61,0.102034,0.071767,0.060440,0.043257,0.039309,0.005167,0.008380,0.001461,0.001832,0.001157,0.006755,0.008682,0.007333,0.024936,0.002021,0.004072,0.018761,0.021065,0.011614,0.001764,0.001736,0.005401,0.012058,0.003264,0.019880,0.006540,0.043257,0.033436,0.001639,0.023925,0.034852,0.001254,0.015497,0.003324,0.001699,0.019145,0.019721,0.102034,0.071767,0.027251,0.004672,0.001286,0.015483,0.002396,0.005372,0.037268,0.015475,0.010001,0.009111,0.027143,0.009656,0.017652,0.027962,0.003187,0.001799,0.011567,0.008130,0.010631,0.017657,0.015933,0.060440,0.002172,0.007501,0.019262,0.016522,0.007316,0.039309,0.015503,0.006915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
trn_x1_emoji = trn_x1_emoji[trn_x1_emoji.columns[-64:]]
np.array(trn_x1_emoji).shape

(15155, 64)

In [ ]:
tst_x1_emoji = pd.DataFrame(emoji.emoji_prediction(tst_txt_df, maxlen=max_len))
tst_x1_emoji = tst_x1_emoji[tst_x1_emoji.columns[-64:]]
tst_x1_emoji.shape

Tokenizing using dictionary from /content/drive/MyDrive/PCL/DeepMoji/model/vocabulary.json
Loading model from /content/drive/MyDrive/PCL/DeepMoji/model/deepmoji_weights.hdf5.
Model: "DeepMoji"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 128, 256)     12800000    ['input_2[0][0]']                
                                                                                                  
 activation_1 (Activation)      (None, 128, 256)     0           ['embedding[0][0]']              
                                                                                                  
 bi_lstm_0 (Bid

(2094, 64)

### Get stacked models and their predictions

In [ ]:
#!pip install pygraphviz
from tensorflow.keras.utils import plot_model
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, LSTM, Input, concatenate, Bidirectional, Embedding, Dropout, Activation, Attention
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [ ]:
def fit_model(trainX, trainy):
  # define model
  callback = EarlyStopping(monitor='val_loss', patience=5)
  input1 = Input(shape=(trn_x_encoded1.shape[1],), name="input1")
  input2 = Input(shape=(trn_x_sc1.shape[1],), name="input2")
  input = concatenate([input1, input2], axis=1, name="concat_input")
  x = Embedding(types_num + 4, 128, mask_zero=True, input_length=max_len)(input)
  x = Dropout(0.25)(x)
  x = Activation('tanh')(x)
  lstm = Bidirectional(LSTM(64, dropout=0.25), name="BiLSTM0")(x)
  dense = Dense(32, activation = "relu", name="Dense_layer")(lstm)
  drop = Dropout(0.25)(dense)
  y = Dense(1, activation="sigmoid", name="Output")(drop)
  model1 = Model(inputs= [input1, input2], outputs=y, name="Model1")
  model1.summary()
  model1.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])
  # fit the model
  model1.fit([trn_x_encoded1, trn_x_sc1], trn_y_encoded1, batch_size=32, epochs=100,
           validation_split=0.2, callbacks=[callback], verbose=2)
	
  return model


# save submodels
for i in range(5):
  # fit the model
  model = fit_model(trn_x_encoded_with_senti, trn_y_encoded1)
  filename = 'modelsForStack/model_' + str(i + 1) + '.h5'
  model.save(filename)
  print('>Saved %s' % filename)


# load self-chosen numbers of submodels
def load_submodels(n_models):
  submodels = []
  for i in range(n_models):
    filename = 'modelsForStack/model_' + str(i + 1) + '.h5'
    model = load_model(filename)
    submodels.append(model)
  return submodels


Model: "Model1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input1 (InputLayer)            [(None, 124)]        0           []                               
                                                                                                  
 input2 (InputLayer)            [(None, 4)]          0           []                               
                                                                                                  
 concat_input (Concatenate)     (None, 128)          0           ['input1[0][0]',                 
                                                                  'input2[0][0]']                 
                                                                                                  
 embedding_5 (Embedding)        (None, 128, 128)     1280512     ['concat_input[0][0]']      

In [ ]:
# or we can use the predicted results for non-nn models, 3d flatten into 2d result
def stacked_preds(trn_x, trn_y, tst_x, num_model, tst_y=None):
  stacked_pred = None
  for i in range(num_model):
    model = fit_model(trn_x, trn_y)
    pred = model.predict(tst_x)
    if stacked_pred is None:
      stacked_pred =  pred
    else:
      stacked_pred = np.dstack((stacked_pred, pred))
  print("The shape of stacked_pred:{}".format(stacked_pred.shape))
  stacked_pred = stacked_pred.reshape((stacked_pred.shape[0], stacked_pred.shape[1]*stacked_pred.shape[2]))
  return stacked_pred

In [ ]:
nn5_preds = stacked_preds(trn_x_encoded_with_senti, trn_y_encoded1, tst_x_encoded_with_senti, 5)
nn5_preds = pd.DataFrame(nn5_preds)
nn5_preds.to_csv("trn_5nn_predictions.csv", encoding="utf-8") # change to trn_x in predict

In [ ]:
# use the output of the submodels as an input for the meta model
submodels = load_submodels(5)
print(submodels)

def ensemble_model(submodels):
  # first make the submodels untrainable during the meta model training
  for i in range(len(submodels)):
	  model = submodels[i]
	  for layer in model.layers:
		  # make not trainable
		  layer.trainable = False
		  # rename to avoid 'unique layer name' issue
		  layer._name = 'ensemble_' + str(i+1) + '_' + layer.name
  inputs_submodel = [model.input for model in submodels]
  #input_emoji = Input(shape=(trn_x1_emoji.shape[1],), name='input_emoji')
  print("submodel inputs: {}".format(inputs_submodel))
  outputs_submodel = [model.output for model in submodels]
  concat = concatenate(outputs_submodel, name='concat_submodel_outputs')
  # we will use the emoji predictions as extra embedding
  #concat = concatenate([concat, input_emoji], name = 'concat_emoji')
  print("concat output: {}".format(concat))
  x = Embedding(types_num, 64, mask_zero=True, 
                input_length=concat.shape[1])(concat)
  x = Dropout(0.25)(x)
  x = Bidirectional(LSTM(128, dropout=0.25), name='BiLSTM')(x)
  x = Dense(32, activation='relu', name='Dense_layer')(x)
  x = Dropout(0.25)(x)
  y = Dense(1, activation="sigmoid", name="Output")(x)
  model = Model(inputs=inputs_submodel, outputs = y, name= 'Model_ensemble1')
  # plot graph of ensemble
  #plot_model(model, show_shapes=True, to_file='model_graph.png')
  model.summary()
  model.compile(loss='binary_crossentropy',
                 optimizer=Adam(learning_rate=0.001),
                 metrics=['accuracy'])
  return model

[<keras.engine.sequential.Sequential object at 0x7f2030612050>, <keras.engine.sequential.Sequential object at 0x7f2030d17f50>, <keras.engine.sequential.Sequential object at 0x7f2030cfa110>, <keras.engine.sequential.Sequential object at 0x7f203053e550>, <keras.engine.sequential.Sequential object at 0x7f203061d910>]


In [ ]:
# fit the model and get predicitions
def fit_ensemble_model(ensemble_model, x, y):
  callback = EarlyStopping(monitor='val_loss', patience=5)
  inputs = [x for _ in range(len(ensemble_model.input))]
  #inputs =  inputs + trn_x1_emoji
  ensemble_model.fit(inputs, y, batch_size=50, epochs=100,
           validation_split=0.2, callbacks=[callback], verbose=2)

def predict_ensemble_model(ensemble_model, x):
  inputs = [x for _ in range(len(ensemble_model.input))]
  #inputs = inputs + [tst_x1_emoji]
  return ensemble_model.predict(inputs, verbose=0)

In [ ]:
ensemble_model = ensemble_model(submodels)
fit_ensemble_model(ensemble_model, trn_x_encoded_with_senti, trn_y_encoded1)

submodel inputs: [<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'dense_input')>, <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'dense_2_input')>, <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'dense_4_input')>, <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'dense_6_input')>, <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'dense_8_input')>]
concat output: KerasTensor(type_spec=TensorSpec(shape=(None, 5), dtype=tf.float32, name=None), name='concat_submodel_outputs/concat:0', description="created by layer 'concat_submodel_outputs'")
Model: "Model_ensemble1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 dense_input (InputLayer)       [(None, 128)]        0           []                               
                                                     

In [ ]:
ensemble_pred = predict_ensemble_model(ensemble_model, tst_x_encoded_with_senti)
print("The predicitons of the ensemble model: {}".format(ensemble_pred))
ensemble_pred = [1 if p[0] >= 0.5 else 0 for p in ensemble_pred]
cls_report = classification_report(tst_y_encoded1, ensemble_pred)
f1_ensem = f1_score(tst_y_encoded1, ensemble_pred)
print(cls_report), f1_ensem #0.32 ensemble without senti-score, 0.38 with senti-score

## Functional API Model:

In [ ]:
callback = EarlyStopping(monitor='val_loss', patience=3)
input1 = Input(shape=(trn_x_encoded1.shape[1],), name="input1")
input2 = Input(shape=(trn_x_sc1.shape[1],), name="input2")
input = concatenate([input1, input2], axis=1, name="concat_input")
x = Embedding(types_num + 4, 256, mask_zero=True, input_length=max_len)(input)
x = Dropout(0.25)(x)
x = Activation('tanh')(x)
lstm0 = Bidirectional(LSTM(512, return_sequences=True,dropout=0.25), name="BiLSTM0")(x)
lstm1 = Bidirectional(LSTM(512, return_sequences=True, dropout=0.25), name="BiLSTM1")(lstm0)
concat = concatenate([lstm0, lstm1, x], name="concat_lstm_embeddi")
attn = AttentionWeightedAverage(name="attention_layer")(concat)
dense = Dense(128, activation = "relu", name="Dense_layer")(attn)
drop = Dropout(0.25)(dense)
y = Dense(1, activation="sigmoid", name="Output")(drop)
model1 = Model(inputs= [input1, input2], outputs=y, name="Model1")
model1.summary()
model1.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])
model1.fit([trn_x_encoded1, trn_x_sc1], trn_y_encoded1, batch_size=250, epochs=30,
           validation_split=0.2, callbacks=[callback], verbose=2)

Model: "Model1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input1 (InputLayer)            [(None, 166)]        0           []                               
                                                                                                  
 input2 (InputLayer)            [(None, 4)]          0           []                               
                                                                                                  
 concat_input (Concatenate)     (None, 170)          0           ['input1[0][0]',                 
                                                                  'input2[0][0]']                 
                                                                                                  
 embedding_1 (Embedding)        (None, 170, 256)     2561024     ['concat_input[0][0]']      

In [ ]:
pred1 = model1.predict([tst_x_encoded1, tst_x_sc1])
pred1

In [ ]:
pred_labels = np.array([1 if p[0] >= 0.5 else 0 for p in pred1])
pred_labels

array([0, 1, 0, ..., 0, 0, 0])

In [ ]:
from sklearn.metrics import f1_score, classification_report

In [ ]:
f1_sc = f1_score(tst_y_encoded1, pred_labels, average='binary')
cr = classification_report(tst_y_encoded1, pred_labels)
f1_sc, print(cr) #0.578603 macro

('              precision    recall  f1-score   support\n\n           0       0.93      0.91      0.92      1895\n           1       0.29      0.35      0.32       199\n\n    accuracy                           0.86      2094\n   macro avg       0.61      0.63      0.62      2094\nweighted avg       0.87      0.86      0.86      2094\n',
 0.6180386984523426)

# Task 2

## Load data:

In [ ]:
from ast import literal_eval

In [ ]:
trndf2 = pd.read_csv("trndf2.csv")
tstdf2 = pd.read_csv("tstdf2.csv")
trndf2.label = trndf2.label.apply(literal_eval)
tstdf2.label = tstdf2.label.apply(literal_eval)
tstdf2

,par_id,text,country,keyword,label
0,4046,we also know that they can benefit by receivin...,us,hopeless,"[1, 0, 0, 1, 0, 0, 0]"
1,1279,pope francis washed and kissed the feet of mus...,ng,refugee,"[0, 1, 0, 0, 0, 0, 0]"
2,8330,many refugees do n't want to be resettled anyw...,ng,refugee,"[0, 0, 1, 0, 0, 0, 0]"
3,4063,"""budding chefs , like """" fred """" , """" winston ...",ie,in-need,"[1, 0, 0, 1, 1, 1, 0]"
4,4089,"""in a 90-degree view of his constituency , one...",pk,homeless,"[1, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...
432,8638,"gregory chen , director of government relation...",nz,immigrant,"[0, 0, 0, 0, 0, 0, 0]"
433,8639,foreign tourists rest on the floor while waiti...,ph,homeless,"[0, 0, 0, 0, 0, 0, 0]"
434,8640,NaN,ke,migrant,"[0, 0, 0, 0, 0, 0, 0]"
435,8641,approve name-change or face ' hopelessness ' :...,ph,hopeless,"[0, 0, 0, 0, 0, 0, 0]"


In [ ]:
trn_x2 = trndf2["text"]
tst_x2 = tstdf2["text"]
trn_y2 = trndf2["label"]
tst_y2 = tstdf2["label"]
for i, t in enumerate(tst_x2):
  if type(t) is float:
    print(i, t)
tst_x2.loc[434] = ''
tst_x2.loc[434]

434 nan


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


''

In [ ]:
trn_x_encoded2, tst_x_encoded2, max_len2 = tok_vec(types_num, trn_x2, tst_x2)
trn_x_encoded2.shape, tst_x_encoded2.shape, max_len2

((1032, 132), (437, 132), 132)

In [ ]:
trn_y_encoded2 = np.array([list(l) for l in trn_y2.values])
tst_y_encoded2 = np.array([list(l) for l in tst_y2.values])
trn_y_encoded2, tst_y_encoded2.shape

(array([[1, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 1, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]), (437, 7))

In [ ]:
#class_weights2 = class_weight.compute_class_weight(class_weight='balanced', 
                                                   #classes=np.unique(trn_y_encoded2), y=trn_y_encoded2)

In [ ]:
%ls
!pip install vaderSentiment
!pip install flair
from sentiment_scoring import senti_score
from scipy.sparse import hstack, csr_matrix
trn_x_sc2 = np.array(senti_score(trn_x2, "vader"))
tst_x_sc2 = np.array(senti_score(tst_x2, "vader"))
trn_x_sc2.shape, tst_x_sc2

backtranslation_subtask2/         rebuilddata.py
data1                             sentiment_scoring.py
data2.pickle                      ten-langs-regex-cleaned-binary-label.csv
dev_semeval_parids-labels.csv     thePoorerTheMerrier/
dontpatronizeme_categories.tsv    tptm.csv
dontpatronizeme_pcl.tsv           train_semeval_parids-labels.csv
dont_patronize_me.py              trndf1.csv
emoji_feature_vectors_trn_x1.csv  trndf2.csv
emoji_feature_vectors_tst_x1.csv  trn_encoded_with_senti_df1
pcl.ipynb                         trn_x1_augmented
pcl-task1-augment-revised.ipynb   trn_x1_original
pcl-task2-augment.ipynb           tstdf1.csv
__pycache__/                      tstdf2.csv
README.txt                        tst_x1_original
     |████████████████████████████████| 125 kB 5.1 MB/s 
     |████████████████████████████████| 322 kB 5.0 MB/s 
     |████████████████████████████████| 64 kB 2.2 MB/s 
     |████████████████████████████████| 19.7 MB 6.2 MB/s 
     |███████████████████████████████

((1032, 4), array([[ 0.    ,  0.804 ,  0.196 ,  0.9515],
        [ 0.    ,  0.934 ,  0.066 ,  0.3818],
        [ 0.187 ,  0.813 ,  0.    , -0.3009],
        ...,
        [ 0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.313 ,  0.687 ,  0.    , -0.6249],
        [ 0.    ,  0.877 ,  0.123 ,  0.4215]]))

In [ ]:
callback = EarlyStopping(monitor='loss', patience=5)
input1 = Input(shape=(trn_x_encoded2.shape[1],), name="input1")
input2 = Input(shape=(trn_x_sc2.shape[1],), name="input2")
input = concatenate([input1, input2], axis=1, name="concat_input")
x = Embedding(types_num+4, 256, mask_zero=True, input_length=max_len)(input)
x = Bidirectional(LSTM(512, dropout=0.25), name="BiLSTM")(x)
x = Dense(128, activation = "relu", name="Dense_layer")(x)
x = Dropout(rate=0.25)(x)
y = Dense(7, activation="sigmoid", name="Output")(x)
model2 = Model(inputs= [input1, input2], outputs=y, name="Model2")
model2.summary()
model2.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])
model2.fit([trn_x_encoded2, trn_x_sc2], trn_y_encoded2, batch_size=32, epochs=100,
           validation_split=0.2, callbacks=[callback], verbose=2)

Model: "Model2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input1 (InputLayer)            [(None, 132)]        0           []                               
                                                                                                  
 input2 (InputLayer)            [(None, 4)]          0           []                               
                                                                                                  
 concat_input (Concatenate)     (None, 136)          0           ['input1[0][0]',                 
                                                                  'input2[0][0]']                 
                                                                                                  
 embedding (Embedding)          (None, 136, 256)     1281024     ['concat_input[0][0]']      

In [ ]:
pred2 = model2.predict([tst_x_encoded2, tst_x_sc2])

In [ ]:
pred2[0]

In [ ]:
pred2_binary = []
for p in pred2:
  p = [1 if i > 0.5 else 0 for i in p]
  pred2_binary.append(p)
pred2_binary

In [ ]:
f1_score2 = f1_score(tst_y_encoded2, pred2_binary, average=None)
f1_score2